## Introduction

See https://github.com/antoineeripret/search-console-decay for the full explanation on how to use this Notebook. Should you have any issue using it, please raise an issue directly in the GitHub repository. 

## Project Setup

In [0]:
#Install or load https://github.com/joshcarty/google-searchconsole. Shoutout to @joshcarty for this amazing library !!  
try:
  import searchconsole
except:
  !pip install git+https://github.com/joshcarty/google-searchconsole
  import searchconsole

#Load other standard libraries that we will use in this notebook. 
import pandas as pd 
import numpy as np 
from datetime import datetime
from datetime import timedelta
import calendar

In [0]:
#Indicate here the property you will use. If you are not sure which ones you can use, just run the folliwng line and pick one. 
#webproperties = [element for element in webproperties if account[element].permission != 'siteUnverifiedUser']

selected_property = 'https://www.liligo.fr/'

#We do not want to analyze partial months, hence enter here teh last day of the last month. If we are the 4th of July, we should have 
# '2020-06-30' here. Format is YYYY-MM-DD. 

last_day_of_last_month= '2020-04-30'

In [0]:
#Create the account connection. Please note that: 
# 1. The flow argument is mandatory to use Josh Carty's library in Google Colab 
# 2. You will have to connect just once with this try / except structure. See https://github.com/joshcarty/google-searchconsole for the explanation.  
try:
	account = searchconsole.authenticate(flow="console",client_config='/content/drive/My Drive/Colab Notebooks/search-console/config/client-secret.json',credentials='/content/drive/My Drive/Colab Notebooks/search-console/config/credentials.json')

except:
	account = searchconsole.authenticate(flow="console",client_config='/content/drive/My Drive/Colab Notebooks/search-console/config/client-secret.json',serialize='/content/drive/My Drive/Colab Notebooks/search-console/config/credentials.json')

In [0]:
# Retrieve the top content from your GSC data. You can add more filters if you have to. 
# Based on Pareto rule, you shouldn't need to retrieve more than 100 contents for your first analysis. 
# Moreover, retrieving 100 URLs could be quite long using the API ;) 

#List of URLs we just mentioned 
urls = account[selected_property].query.filter('page','magazine','contains').range('today', days=-1000).dimension('page').limit(25).get().to_dataframe()['page']

#We initiate an empty DataFrame we will use later on 
merged_gsc_data = pd.DataFrame()

#Function to normalize months: if the month number has just one digit, we add a 0 at the beginning 
def normalizemonth(x):
	if len(str(x))==1:
		return '0'+str(x)
	else:
		return str(x) 

#We start a loop
i=0
#We loop our urls list that is storing our contents urls 
for url in urls[0:5]:
  #We print i just to follow the advance of our cell inside Colab 
  print(i)
  # We retrieve all data available until the last day of last month. 
  # We add -1000 as the days arguments to be sure to retrieve all data available. 
  gsc_data = account[selected_property].query.filter('page',url,'equals').range(last_day_of_last_month,days=-1000).dimension('page','date').get().to_dataframe()
	#Convert date to datetime object 
  gsc_data['date'] = pd.to_datetime(gsc_data['date'],format='%Y-%m-%d')
  #Extract month and normalize it using our custom function 
  gsc_data['month'] = gsc_data['date'].dt.month.apply(normalizemonth)
  #We build a yearMonth column  
  gsc_data['yearMonth'] = gsc_data['date'].dt.year.astype(str)+gsc_data['month']
  #We group data per yearMonth to analyze data monthly and not daily, which is better  
  gsc_data = gsc_data.groupby(['yearMonth']).sum()
  #Remove useless columns and reset index 
  gsc_data = gsc_data[['clicks','impressions']].reset_index()
  #Add a "false" day (the first of the month) to our date column to be able to calculate time deltas between two months 
  gsc_data['date'] = gsc_data['yearMonth']+'01'
  #Convert this field to datetime 
  gsc_data['date'] = pd.to_datetime(gsc_data['date'],format='%Y%m%d')
  #Add the URL in our dataFrame
  gsc_data['url'] = url
  #Add this data to the DF we will be using later on 
  merged_gsc_data = pd.concat([merged_gsc_data,gsc_data])
  i+=1


#Function to retrieve the peak month of any given content 
def get_peak(x):
	df = merged_gsc_data[merged_gsc_data['url']==x].sort_values(by='clicks',ascending=False)
	peak = df['yearMonth'].iloc[0]
	peak = peak+'01'

	return peak

#Function to retrieve the peak clicks of any given content 
def get_max(x):
	df = merged_gsc_data[merged_gsc_data['url']==x].sort_values(by='clicks',ascending=False)
	max_value = df['clicks'].max()

	return max_value


#Retrieve the peak in our merged DF
merged_gsc_data['peak'] = merged_gsc_data['url'].apply(get_peak)
#Transform the peak to a date object
merged_gsc_data['peak'] = pd.to_datetime(merged_gsc_data['peak'],format='%Y%m%d')
#Retrieve the max in our merged DF
merged_gsc_data['max'] = merged_gsc_data['url'].apply(get_max)
#Calculate the loss % between any given month and the peak 
merged_gsc_data['%_lost'] = ((1-(merged_gsc_data['clicks']/merged_gsc_data['max']))*100).astype(int)
#Calculate net loss between any given month and the peak
merged_gsc_data['lost'] = merged_gsc_data['clicks'] - merged_gsc_data['max']
#Calculate the number of months since the peak (not used but you may need this info)
merged_gsc_data['n_since_peak'] = (merged_gsc_data['date'].dt.year - merged_gsc_data['peak'].dt.year)*12 + (merged_gsc_data['date'].dt.month - merged_gsc_data['peak'].dt.month)

#Funtion to get the current loss vs peak 
def get_current_loss_vs_peak(x):
  df = merged_gsc_data[merged_gsc_data['url']==x]
  value = df[df['date']==df['peak']]['clicks'].sum()-df[df['date']==df['date'].max()]['clicks'].sum()
  return value

#Functon to get the current % loss vs peak 
def get_current_loss_percent_vs_peak(x):
  df = merged_gsc_data[merged_gsc_data['url']==x]
  value = (df[df['date']==df['date'].max()]['clicks'].sum() - df[df['date']==df['peak']]['clicks'].sum())*100/(df[df['date']==df['peak']]['clicks'].sum())

  return value


# Apply both function to create new columns
merged_gsc_data['current_percent_loss'] = merged_gsc_data['url'].apply(get_current_loss_percent_vs_peak)
merged_gsc_data['current_loss'] = merged_gsc_data['url'].apply(get_current_loss_vs_peak)

# Transform data to get our main decay summary data
decay_raw = merged_gsc_data.groupby('url').aggregate({'n_since_peak':np.max,'current_loss':np.max,'current_percent_loss':np.max})
decay_raw = decay_raw.sort_values(by='current_loss',ascending=False)


In [0]:
#Print first rows to be sure that data are accurate 
decay.head()

In [0]:
#Create an empty DataFrame for our final result 
final_data = pd.DataFrame()

#Loop urls in our decay_raw DF
for url in decay.index.to_list()[0:2]:
  #Get the peak date + 1 month as search console library will need the end date and the number of days backwards 
  #Retrieve peak month using our already used function
  peak = get_peak(url)
  #Convert it to datetime
  peak = datetime.strptime(peak,"%Y%m%d")
  #Retrieve the number of months in this particular month thanks to the calendar library
  days_in_month = calendar.monthrange(peak.year, peak.month)[1]
  #Add this number of days 
  date_for_sc = peak + timedelta(days=days_in_month)
  #Convert to string 
  date_for_sc = date_for_sc.strftime("%Y-%m-%d")
  #Retrieve metrics per query for the peak month and the last month
  data_peak = account[selected_property].query.filter('page',url,'equals').range(date_for_sc,days=-days_in_month).dimension('query').limit(100).get().to_dataframe()
  data_current = account[selected_property].query.filter('page',url,'equals').range('2020-04-30',days=-30).dimension('query').limit(100).get().to_dataframe()
  
  #Merge both DataFrames on the query column 
  append = data_peak.merge(data_current,on='query',how='outer',suffixes=('_peak','_current'))
  #Add the URL to the merged dataframe
  append['url'] = url
  #Add data to our final_data DataFrame that we created earlier 
  final_data = pd.concat([final_data,append])

#Check first rows
final_data.head()

In [0]:
#Function to use the cumsum() 
def get_cum_sum(x):
  df = final_data[final_data['url']==x]
  value = df['clicks_diff']*100/df['clicks_diff'].sum()
  return value 

In [0]:
#Replace NaN caused by our merge by 0. Otherwise, we won't be able to visualize lost keywords for instance. 
final_data = final_data.fillna(0)
#Calculate clicks diff between peak month and current month
final_data['clicks_diff'] = final_data['clicks_peak'] - final_data['clicks_current']
#Order DF by click diff 
final_data = final_data.sort_values(by='clicks_diff',ascending=False)
#Remove useless columns 
final_data = final_data[['url','query','clicks_peak','clicks_current','clicks_diff',]]
#Remove any keyword with a positive difference, as twe are not interested by these ones here 
final_data = final_data[final_data['clicks_diff']>0]
#Calculate the % of the total diff per keyword 
final_data['%_diff'] = (final_data['clicks_diff']*100/final_data['clicks_diff'].sum())
#Create a cumsum() column to visualize how to apply Pareto principles to our results 
final_data['cum_%_diff'] = final_data['%_diff'].cumsum()

In [0]:
#Final DF. You can save it in .csv (pd.to_csv()) or send it to Sheets using gspread library for instance. 
final_data